In [ ]:
import shlex, subprocess, time

import pandas as pd
import glob

import runconfig

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
config_dict = runconfig.get_config_dict()
dataset_name = config_dict['dataset_name']
outlier_fraction = config_dict['outlier_fraction']
num_clients_per_class = config_dict['num_clients_per_class']
association_threshold = config_dict['association_threshold']

num_class = 10

l_ideal = []
for p1 in range(num_class):
    l2 = []
    for p2 in range(num_clients_per_class):  
        l2.append(str(p1)+"_"+str(p2))
    l_ideal.append(l2)
l_ideal
dataset_name

In [ ]:
communities_file_name = dataset_name + str(num_clients_per_class) + '_communities.txt'
with open('data/'+communities_file_name) as file:
    lines = file.readlines()
    l_communities = [sorted(line.strip().replace(' ', '').split(',')) for line in lines]

for i in l_communities:
    print(i)

In [ ]:
%%capture

liss = l_communities; out_fold = 'communities' 
#liss = l_ideal; out_fold = 'ideal'  # run this for ideal case

address = 5000 # is actually a port. Server and clients of the same group have the same port
for num,i in enumerate(liss[0:]):
    print(str(i))
    ll = ['./fed_start.sh', dataset_name, str(address+num), str(num_clients_per_class), out_fold] + i
    as_string = " ".join(ll)
    subprocess.check_call(as_string, shell=True)


In [ ]:
# put the evaluation results together
def get_df_from_csvs(pth, out_fold):
    files = glob.glob(pth+"output_"+out_fold+"/"+dataset_name+"/*.csv")
    df = pd.DataFrame()
    for f in files:
        csv = pd.read_csv(f,index_col=[0])
        df = pd.concat([df,csv])
        df = df.sort_values('dataset').reset_index(drop=True)
    df = df.add_suffix('_'+out_fold).reset_index(drop=True)
    return df

pth = 'out'+str(num_clients_per_class)+'/'
df1 = get_df_from_csvs(pth, 'local')
df2 = get_df_from_csvs(pth, 'communities')
df3 = get_df_from_csvs(pth, 'ideal')
pd.concat([df1,df2,df3],axis=1)

In [ ]:
metric = 'f1in'
metric = 'f1out'
#metric = 'acc'
metric = 'aucroc'
pd.concat([df1,df2,df3],axis=1)
pd.concat([df1,df2,df3],axis=1)[[metric+'_local',metric+'_communities',metric+'_ideal']].reset_index(drop=True).style.highlight_max(color = 'lightgreen', axis = 1)
pd.concat([df1,df2,df3],axis=1)[[metric+'_local',metric+'_communities',metric+'_ideal']].reset_index(drop=True).mean()

In [ ]:
df2

In [ ]:
df3['inlier'] = df3['dataset_ideal'].str.split('_', 1, expand=True)[0] # only need to add inlier column
pd.concat([df1,df2,df3],axis=1)[['inlier',metric+'_local',metric+'_communities',metric+'_ideal']].groupby('inlier').mean().style.highlight_max(color = 'lightgreen', axis = 1)